# Automatic differentiation with `torch.autograd`
When training neural networks, the most frequently used alghrithm is **back propagation**.
In this algorithm, parameters (model weights) are adjusted according to the gradient of the loss  function with respect to the given paramter.

To compute those gradients, PyTorch has built-in differentiation engine called `torch.autograd`.
It supports automatic computation of gradient for any computational graph.

Consider the simplest one-layer neural network, with input `x`, parameters `w` and `b`, and some loss function.
It can be defined in PyTorch in the following manner:

ニューラルネットワークの訓練の時に，最もよく使うアルゴリズムは**誤差逆伝播法**である．
このアルゴリズムにおいては，与えられたパラメータに対する損失関数の勾配に従ってパラメータは調整される．

これらの勾配を計算するために，PyTorchには`torch.autograd`と言われる組み込みの微分エンジンがある．
それによって，あらゆる計算グラフに対しても，自動的に勾配を計算することができる．

最も単純なニューラルネットワーク，入力が`x`，パラメータが`w`，`b`，ある損失関数の1層のニューラルネットワークを考える．
PyTorchでは，次の文法に従って定義される．

In [1]:
import torch
print(f'torch version: {torch.__version__}')

x = torch.ones(5) # input tensor
y = torch.zeros(3) # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print(f'x: {x}')
print(f'y: {y}')
print(f'w: {w}')
print(f'b: {b}')
print(f'z: {z}')
print(f'loss: {loss}')

torch version: 1.8.1
x: tensor([1., 1., 1., 1., 1.])
y: tensor([0., 0., 0.])
w: tensor([[-0.1022,  0.3488, -0.0435],
        [ 0.6768,  0.7786,  0.0043],
        [-1.3720,  0.7665, -1.8001],
        [-0.1794, -1.7732,  0.8966],
        [-0.8091,  0.1308,  0.9481]], requires_grad=True)
b: tensor([-1.3612, -2.4295, -0.8523], requires_grad=True)
z: tensor([-3.1471, -2.1780, -0.8470], grad_fn=<AddBackward0>)
loss: 0.16871707141399384


## Tensors, Functions and Computational graph
This code defines the following computational graph:
![computational graph](graph/autograd_computational_graph.png)

In this network, `w` and `b` are parameters, which we need to optimize.
Thus, we need to be abel to compute the gradients of loss function with respect to those variables.
In order to do that, we set the `requires_grad` property of those tensors.

このコードは次の計算グラフで定義される．

このネットワークにおいて，`w`と`b`がパラメータで，最適化が必要である．
それゆえ，これらの変数に対する損失関数の勾配を計算することが必要である．
そのためにも，これらのtensorの`requires_grad`プロパティをセットする必要がある．

#### Note
You can set the value of `requires_grad` when creating a tensor, or later by using `x.required_grad_(True)` method.

 tensorを定義した時に，`requires_grad`をセットする．もしくは後で後から`x.required_gard_(True)`メソッドを用いる．

A function that we apply to tensors to construct computational graph is in fact an object of class `Function`.
This object knows how to compute the function in the forward direction, and also how to compute its derivative during the backward propagation step.
A reference to the backward propagation function is stored in `grad_fn` property of a tensor.
You can find more information of `Function` [in the documentation](https://pytorch.org/docs/stable/autograd.html#function).

計算グラフを作成するためにtensorに適用する関数は，実際に`Function`クラスのオブジェクトである．
このオブジェクトは順方向の関数を計算する方法と，逆伝播における微分の計算方法を備えている．
逆伝播関数への参照はtensorの`grad_fn`プロパティに保存されている．
詳細を知りたい場合は，`Function`を見ること．

In [2]:
print('Gradient function for z = ', z.grad_fn)
print('Gradient function for loss = ', loss.grad_fn)

Gradient function for z =  <AddBackward0 object at 0x105efad30>
Gradient function for loss =  <BinaryCrossEntropyWithLogitsBackward object at 0x105efab80>


## Computing Gradients
To optimize weights of parameters in the neural network, we need to compute the derivatives of our loss function iwth respect to parameters, namely, we need $\dfrac {\partial loss}{\partial w}$ and $\dfrac{\partial loss}{\partial b}$ undersome fixed values of `x` and `y`.
To compute those derivatives, we call `loss.backward()`, and then retrieve the values from `w.grad` and `b.grad`.

ニューラルネットワークのパラメータの係数を最適化するために，パラメータに対する損失関数の微分を計算する必要があり，
つまり，ある`x`と`y`の固定値に元での$\dfarc{\partial y_}{\partial x_}$が必要である．
これら微分を計算するために，`loss.backward()`を呼び出し，その後，`w.grad`と`b.grad`から値を得ることができる．

In [3]:
print(w.grad)
print(b.grad)
loss.backward()
print(w.grad)
print(b.grad)

None
None
tensor([[0.0137, 0.0339, 0.1000],
        [0.0137, 0.0339, 0.1000],
        [0.0137, 0.0339, 0.1000],
        [0.0137, 0.0339, 0.1000],
        [0.0137, 0.0339, 0.1000]])
tensor([0.0137, 0.0339, 0.1000])


#### NOTE
- We can only obtain the `grad` properties for the leaf nodes of the computational graph, which have `requires_grad` property set to `True`. For all other nodes in our graph, gradients will not be available.
- We can only perform gradient calculations using `backward` once on a given graph, for performance reasons. If we need to do several `backward` calls on the same graph, we need to pass `retain_graph=True` to the `backward` call.

- `requires_grad`プロパティを`True`にセットした計算グラフのノードの`grad`プロパティのみを得ることができる．計算グラフの他のノードは勾配を使用できない．
- パフォーマンス上の理由から，与えられたグラフに対する`backward`を用いた勾配の計算は一度だけである．もし同じ計算グラフに対して何回か`backward`を実行したいなら，`backward`の呼び出しに`retain_graph=True`を渡す必要がある．

## Disabling Gradient Tracking
By default, all tensors with `requires_grad=True` are tracking their computational history and support gradient computation.
However, there are some cases when we do not need to do that, for example, when we have trained the model and just want to apply it to some input data, i.e. we only want to do forward computations through the network.
We can stop tracking computations by surrounding our computation code with `torch.no_grad()` block:

デフォルトでは，`requires_grad=True`を持った全てのtensorは計算過程を追跡されていて，勾配計算をサポートしている．
しかしながら，そのようなことが必要ない場合，例えばモデルを訓練して同じ入力データに適用したい場合，つまりネットワークの順伝播しか必要ない場合がある．
`troch.no_grad()`ブロック内に計算コードを書くことで，計算の追跡を止めることが可能である．

補足；
テストデータやバリデーションデータに対しても使用する．

In [4]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


Another way to achive the same result is to use the `detach()` method on the tensor:

別の方法として，tensorに対して，`detach()`メソッドを用いる方法もある．

In [5]:
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z.requires_grad)
print(z_det.requires_grad)

True
False


There are reasons you might want to disable gradient tracking:

- To mark some parameters in your neural network at **frozen parameters**. This is a very common scenario for [finetuning a prettained network](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)
- To speed up computations when you are only doing forward pass, because computations on tensors that do not track gradients would be more efficient.

勾配追跡を無効にする理由はおそらく以下の通りである．

- ニューラルネットワークにといて，一部のパラメータを**凍結パラメータ**でマークしたい．これは事前に訓練されたネットワークを微調整する一般的な方法である．
- 順伝播を計算する時に計算速度を上げたい．というのも，勾配を追跡しないtensorの計算は非常に効率的である．

## More on Computational Graphs
Conceptually, autograd keeps a record of data (tensors) and all executed operations (along with the resulting ner tensors) in directed acyclic graph (DAG) consisting of [Functions](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function) objects.
In this DAG, leaves are the inout tensors, roots are the output tensors.
By tracing this graph from roots to leaves, you automatically compute the gradients using the chain rule.

In a forward pass, autograd does two things simultaneously:

- run the requestd operation to compute a resulting tensor
- maintain the operation's _graient function_ in the DAG.

The backward pass kicks off when `.backward()` is called on the DAG root.
`autograd` then:

- computes the gradients from each `.grad_fn`
- accumulates them in the respective tensor's `.grad` attribute
- using the cahin rule, propagates all the way to the leaf tensors.

概念的には，autogradはデータ(tensor)と実行された全ての操作(結果の新しいtensorと共に)を，関数オブジェクトで構成される有向非巡回グラフ(DAG)に保持する．
このDAG内には，葉は入力tensor，根は出力tensorである．
このグラフを根から葉までトレースすることにより，連鎖律をおちいて自動的に勾配を計算することができる．

順伝播において，autogradは同時に二つのことを行なっている．

- 要求された操作を実行して，結果のtensorを計算する
- DAG内で操作の_gradient function_を維持する

`.backward()`がDAGの値で呼ばれると，逆伝播が始まる．`autograd`は

- それぞれの`.grad_fn`から勾配を計算する
- それらの値をそれぞれのtensorの`.grad`アトリビュートに累積していく
- 連鎖律を用いて，葉の方向へ全て伝播していく

#### NOTE
**DAGs are dynamic in PyTorch**
An important thing to note is that the graph is recreated from scatch; 
after each `.backward()` call, autograd starts populating a new graph.
This is exactly what allows you to use control flow statements in your model;
you can cahnge the shape, size and operations at every iteration if needed.

**PyTorchにおけるDAGのダイナミクス**

注意すべき重要な点は，グラフが最初から再生成されることである．
`.backward()`が呼ばれ，autogradは新しいグラフの作成を開始する．
これはまさに，モデルで制御フローステートメントを使用できるようにするものである．
必要に応じてイテレーション毎に，形やサイズ，操作を変更することが可能である．

## Optional Reading: Tensor Gradients and Jacobian Products
In many cases, we have a scalar loss function, and we need to compute the gradient with respect to some parameters.
However, there are cases when the output function is arbitrary tensor.
In this case, PyTorch allows you to compute so-called **Jacobian product**, and not the actual gradient.

For a vector function $\vec{y} = f(\vec{x})$, where $\vec{x} = \langle x_1, ..., x_n \rangle$ and 
$\vec{y} = \langle y_1, ... y_m \rangle$, agradient of $\vec{y}$ with respect to $\vec{x}$ is given by **Jacobian matrix**:
$$
J = \left(
    \begin{array}{ccc}
        \dfrac{\partial y_1}{\partial x_1} & \ldots & \dfrac{\partial y_1}{\partial x_n} \\
        \vdots & \ddots & \vdots \\
        \dfrac{\partial y_m}{\partial x_n} & \ldots & \dfrac{\partial y_m}{\partial x_n} \\
    \end{array}
\right)
$$

Instead of computing the Jacobian matrix itself, PyTorch allows you to compute **Jacobian Product** $v^T \cdot J$ for a given input vector $v = (v_1 ... v_m)$.
This is achieved by calling `backward` with $v$ as an argument.
The size of $v$ should be the same as the size of the original tensor, with respect to which we want to compute the product:


多くのケースでは，スカラー損失関数があり，いくつかのパラメータに関して勾配を計算する必要がある．
しかしながら，出力関数が任意のtensorである場合もある．
この場合では，PyTorchを使用すると，実際の勾配ではなく**ヤコビアン積**を計算をする．

ベクトル関数$\vec{y} = f(\vec{x})$に対して，$\vec{x} = \langle x_1, ..., x_n \rangle$， 
$\vec{y} = \langle y_1, ... y_m \rangle$とすると，$\vec{x}$に対する$\vec{y}$は上記の**ヤコビアン行列**で与えられる．
ヤコビアン行列の代わりに，PyTorchではヤコビアン積$v^T \cdot J$を計算する．
$v = (v_1 ... v_m)$で与えられる．
これは$v$を引数として，`backward`を呼び出すことで可能である．
$v$のサイズは，積を計算する元のtensorと同じサイズでなければならない．

In [11]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
print('inp\n', inp)
print('out\n', out)

out.backward(torch.ones_like(inp), retain_graph=True)
print('\nFirst call\n', inp.grad)

out.backward(torch.ones_like(inp), retain_graph=True)
print('\nSecond call\n', inp.grad)

inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print('\nCakk after zeroing gradients\n', inp.grad)

inp
 tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], requires_grad=True)
out
 tensor([[4., 1., 1., 1., 1.],
        [1., 4., 1., 1., 1.],
        [1., 1., 4., 1., 1.],
        [1., 1., 1., 4., 1.],
        [1., 1., 1., 1., 4.]], grad_fn=<PowBackward0>)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Cakk after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


Notice that when we call `backward` for the second time with the same argument, the value of the gradient is different.
This happens because when doing `backward` propagation, PyTorch **accumulates the gradients**, i.e. the value of computed gradients is added to the `grad` property of all leaf nodes of computational graph.
If you want to compute the proper gradients, you need to zero out the `grad` propetry before.
In real-life training an _optimizer_ helps us to do this.

同じ引数を使用して2回目に`backward`を使用すると，勾配が異なることに注意が必要である．
これは，`backward`伝播をした時に，PyTorchは勾配を累積する，つまり，計算した勾配の値は計算グラフの全てのノードの`grad`プロパティに加算されていくためである．
適切な勾配を計算するためには，逆伝播する前に，`grad`プロパティを0にする必要がある．
実際のトレーニングでは_optimizer_がkろえを助けてくれる．

#### NOTE
Previously we were calling `backward()` function without parameters.
This is essentially equivalent to calling `backward(torch.tensor(1.0))`, which is a useful way to compute the gradients in case of a scaller-valued function, such as loss during neural network training.

以前はパラメータなしで`backward()`関数を呼び出した．
これは`backward(torch.tensor(1.0))`を呼び出したことと同じであり，ニューラルネットワーク訓練中の損失など，スカラー値関数の勾配を計算する時に便利である．

## Further Reading
- [Autograd Mechanics](https://pytorch.org/docs/stable/notes/autograd.html)